# Рекомендация тарифов

# Задача

Нужно построить модель для задачи классификации способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра»).

Доля правильных ответов должна быть не менее 75%.

Предобработка данных не требуется (она проведена в предыдущей работе).

# План

**1. Обзор данных.**

**2. Разделение данных по выборкам.**

**3. Исследование моделей.**
- Решающее дерево;
- Случайный лес;
- Логистическая регрессия.

**4. Проверка лучшей модели на тестовой выборке.**

**5. Проверка модели на адекватность.**

**6. Выводы.**

## Обзор данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [3]:
print(df.loc[df['is_ultra'] == 1, 'is_ultra'].count() / len(df) * 100)

30.647168637212197


Пропусков в данных нет.

Из всего набора данных только 30% отоносятся к тарифу "Ультра", а значит нашим моделям будет сложнее научиться определять, когда предлагать этот тариф.

## Разделение данных на выборки

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

# Разобьем данные на обучающую, валидационную и тестовую выборки
features_train, features_ost, target_train, target_ost = train_test_split(
   features, target, test_size=0.4, random_state=12345, stratify=target)
features_valid, features_test, target_valid, target_test = train_test_split(
   features_ost, target_ost, test_size=0.5, random_state=12345, stratify=target_ost)

print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)


## Исследование моделей

### Решающее дерево

In [5]:
tree_model = None
best_result = 0
best_depth = 0

for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)

    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
            tree_model = model
            best_result = result
            best_depth = depth

print("Глубина:", best_depth)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Глубина: 8
Accuracy наилучшей модели на валидационной выборке: 0.80248833592535


Лучший показатель accuracy для решающего дерева 0.8 при глубине 8.

### Случайный лес

In [6]:
random_forest_model = None
best_result = 0
best_est = 0
best_depth = 0

for depth in range(1, 10):
    for est in range(1, 25):
        model = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=12345)
        model.fit(features_train, target_train)
        result = model.score(features_valid, target_valid)
        if result > best_result:
            random_forest_model = model
            best_result = result
            best_est = est
            best_depth = depth

print('Кол-во деревьев:', best_est)
print('Глубина:', best_depth)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Кол-во деревьев: 22
Глубина: 8
Accuracy наилучшей модели на валидационной выборке: 0.8149300155520995


Лучший показатель accuracy для случайного леса 0.82 при кол-ве деревьев 22 и глубине 8.

### Логистическая регрессия

In [7]:
regression_model = None
best_result = 0
best_iter = 0

for i in range(100, 1000, 100):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=i) 
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
            regression_model = model
            best_result = result
            best_iter = i
            
print('Кол-во итераций:', best_iter)
print("Accuracy наилучшей модели на валидационной выборке:", best_result)

Кол-во итераций: 100
Accuracy наилучшей модели на валидационной выборке: 0.7387247278382582


Лучший показатель accuracy для логистической регресии 0.73 с количеством итераций 100.

Лучший результат на валидационной выборке показал случайный лес с accuracy 0.82, количеством деревьев 22 и глубиной 8.

## Проверка лучшей модели на тестовой выборке

In [8]:
#Проверим модель на тестовой выборке.
final_features_train = pd.concat([features_train, features_valid])
final_target_train = pd.concat([target_train, target_valid])
model = RandomForestClassifier(random_state=12345, n_estimators=22, max_depth=8)
model.fit(final_features_train, final_target_train)
random_forest_model_result = random_forest_model.score(features_test, target_test)

print('Случайный лес:', random_forest_model_result)

Случайный лес: 0.8009331259720062


На тестовой выборке модель случайного леса показала точность в 0.8.

## Проверка модели на адекватность

In [9]:
dummy_cl = DummyClassifier(strategy='most_frequent', random_state=1)
dummy_cl.fit(features_train, target_train)
dummy_cl.predict(features_test)
print('Точность константной модели:', dummy_cl.score(features_test, target_test))

Точность константной модели: 0.6936236391912908


## Выводы

В ходе работы мы построили 3 различных модели для решения задачи классификации:
1. Модель решающего дерева показала точность 0,8 при глубине 8.
2. Модель случайного леса показала точность 0,82 при кол-ве деревьев 22 и глубине 8.
3. Модель логистической регрессии показала точность  0,73 с количеством итераций 100.

Для проверки на тестовой выборке мы выбрали лучшую модель: модель случайного леса. Ее точность на тестовой выборке составила 0,8. Для нас требовалось построить модель с точность не менее 0,78. Наша модель соответствует этому требованию.

Также мы проверили нашу модель на адекватность, построив константную модель, которая показала точность 0,69. Наша модель случайного леса показывает лучший результат.